In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing, svm

train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

train_df[train_df['target']==0]['text'].values[2]

In [ ]:
train_df[train_df['target']==1]['text'].values[2]

In [ ]:
count_vectorizer = feature_extraction.text.CountVectorizer()
train_vectors = count_vectorizer.fit_transform(train_df["text"])
test_vectors = count_vectorizer.transform(test_df["text"])

In [ ]:
clf = svm.SVC()
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

In [ ]:
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import numpy
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from statistics import Mean

X = train_vectors.todense()
y = train_df['target']
#X_test = test_df['text']
#y_test = test_df['target']

skf = StratifiedKFold(n_splits=7, shuffle=True)

acuracia = []
for train_index, test_index in skf.split(X,y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    
    # create model
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation='relu',))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    #opt = SGD(learning_rate=0.001)
    opt = Adam(learning_rate=0.01)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

    history = model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0, callbacks=[es])

    _, test_acc = model.evaluate(X_test, y_test, verbose=0)
    acuracia.append(test_acc)
    
#print('Acurácia: ', mean(acuracia),'(', stdev(acuracia),')')

In [ ]:
_, test_acc = model.evaluate(X_test, y_test, verbose=0)
acuracia.append(test_acc)
acuracia

In [ ]:
x_new = test_vectors.todense()
y_new = model.predict_classes(x_new)
y_new

In [ ]:
submission = pd.DataFrame(columns=['id','target'])
submission['id'] = test_df['id']
submission['target'] = y_new
submission.to_csv('submission.csv',index=False)